# SSQM v2.0
## Using NBA Shooting Data: Considers shot type, closest defender distance and touch time
## Bin shots by filtering each condition combination and then use that for SSQM

In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
# from nba_api.stats.endpoints import playerdashptshots, leaguedashplayerbiostats

pd.options.mode.chained_assignment = None

data_DIR1 = "../data/box/"
data_DIR2 = "../data/shots/"
csv_export_DIR = "C:/Users/pansr/Documents/repos/csv/"
fig_DIR = "../figs/shots/"

In [ ]:
season = 2024
df = pd.read_parquet(data_DIR2 + f"NBA_Shots_T_{season}_All.parquet")

In [ ]:
df

In [ ]:
df = df[["TEAM_ID","TEAM_NAME","TEAM_ABBREVIATION","FGM","FGA","FG2M","FG2A","FG3M","FG3A", 'general_range', 'closest_def', 'touch_time']]
df_avg = df.groupby(['general_range', 'closest_def', 'touch_time']).sum()
df_avg = df_avg.drop(columns= ["TEAM_ID","TEAM_NAME","TEAM_ABBREVIATION"])
df_avg["xFG2"] = df_avg["FG2M"]/df_avg["FG2A"]
df_avg["xFG3"] = df_avg["FG3M"]/df_avg["FG3A"]
df_avg1 = df_avg.copy().reset_index()
df_avg = df_avg.drop(columns =["FGM","FGA","FG2M","FG2A","FG3M","FG3A"])
df_avg = df_avg.reset_index()

In [ ]:
shots = pd.merge(df,df_avg,on=['general_range', 'closest_def', 'touch_time'])
shots["FG2_PCT"] = shots["FG2M"]/shots["FG2A"]
shots["FG3_PCT"] = shots["FG3M"]/shots["FG3A"]
shots = shots.replace([np.inf, -np.inf], np.nan)
shots = shots.fillna(0)
shots["PTS2"] =  (2*shots["FG2A"]*shots["FG2_PCT"]).round(2)
shots["PTS3"] =  (3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["PTS"] =  (2*shots["FG2A"]*shots["FG2_PCT"] + 3*shots["FG3A"]*shots["FG3_PCT"]).round(2)
shots["xPTS2"] = (2*shots["FG2A"]*shots["xFG2"]).round(2)
shots["xPTS3"] = (3*shots["FG3A"]*shots["xFG3"]).round(2)
shots["xPTS"] = (2*shots["FG2A"]*shots["xFG2"] + 3*shots["FG3A"]*shots["xFG3"]).round(2)

In [ ]:
# shots.query("PLAYER_NAME == 'LeBron James'")

In [ ]:
dfla = pd.DataFrame(df_avg1.iloc[:,3:9].mean()).T
dfla["FG2_PCT"] = dfla["FG2M"]/dfla["FG2A"]
dfla["FG3_PCT"] = dfla["FG3M"]/dfla["FG3A"]
dfla["PTS2"] =  (2*dfla["FG2A"]*dfla["FG2_PCT"]).round(2)
dfla["PTS3"] =  (3*dfla["FG3A"]*dfla["FG3_PCT"]).round(2)
dfla["PTS"] =  (2*dfla["FG2A"]*dfla["FG2_PCT"] + 3*dfla["FG3A"]*dfla["FG3_PCT"]).round(2)

In [ ]:
fg = (shots
    .groupby(['TEAM_NAME'])[['FGM', 'FGA', 'PTS', 'xPTS']]
    .agg({'FGM': ["sum"], 'FGA': ["sum"], 'PTS': ["sum"], 'xPTS': ["sum"]}))
fg.columns = ['FGM', 'FGA', 'PTS', 'xPTS']
fg['eFG'] = np.round(fg['PTS']/fg['FGA']/2, 3)
fg['xeFG'] = np.round(fg['xPTS']/fg['FGA']/2, 3)
fg['Shot_Making'] = np.round((fg['PTS'] - fg['xPTS'])/fg['FGA'], 3)
fg["Points Added"] = fg["PTS"]-fg["xPTS"]
fg = fg.reset_index()

In [ ]:
df_teams = pd.read_csv("../data/" + "NBA_teams_colors_logos.csv")
df_teams["Team"] = df_teams["nameTeam"]
df_teams = df_teams[["Team","colorsTeam","urlThumbnailTeam"]]

In [ ]:
fg = fg.sort_values("Shot_Making",ascending=False).reset_index(drop=True)

In [ ]:
fg["Team"] = fg["TEAM_NAME"]
fg["team_name"] = fg["TEAM_NAME"]
tcat = fg["TEAM_NAME"].to_list()
tcat.reverse()
fgp = add_tinfo(fg)
fgp["Team"] = pd.Categorical(fgp["TEAM_NAME"] , categories=tcat)

In [ ]:
p = (
    ggplot(fgp)
    + aes(x="Team",y="Shot_Making",fill="colorsTeam",image="image")
    # + geom_col(show_legend=False, position=position_stack(reverse=True),color="white",alpha=0.7)
    + geom_col(alpha=0.7,show_legend=False)
    + geom_image(aes(y=0),size=0.06)
    # + scale_y_continuous(labels=percent_format())
    + scale_color_identity(aesthetics=["fill"])
    # + coord_cartesian(ylim=[0.53,0.57])
    # + ylim([0.53,0.57])
    + coord_flip()#ylim=[0.53,0.56]
    + theme_idv
    + labs(
        y="Shot Making: PPS above expected",
        x="Team",
        title=f"NBA Best Shot Making Teams {get_ss(season)}",
        subtitle="According to SSQM 2.0: defender distance, shot type and touch time",
    )
    + pnba
    + theme(
        figure_size=(10,10),
        axis_text_y=element_text(size=14),
        axis_text_x=element_text(size=12),
        axis_title_x=element_text(size=16),
        axis_title_y=element_blank(),
        plot_title=element_text(size=20),
    )
)
p.save(fig_DIR + "team_shot_making.png", verbose=False,dpi=300)
p

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
fgp.columns

In [ ]:
%%R -i fgp
results <- fgp
library(tidyverse)
library(ggimage)
library(ggrepel)
theme_owen <- function() {
  theme_minimal(base_size = 16, base_family = "Consolas") %+replace%
    theme(
      panel.grid.minor = element_blank(),
      plot.background = element_rect(fill = "ghostwhite", color = "ghostwhite")
    )
}
p <- ggplot(
  results,
  aes(x = xeFG, y = Shot_Making)
) +
  # geom_point(aes(size = aNRtg_Rank)) +
  # scale_y_reverse() +
  geom_hline(aes(yintercept = mean(eFG)), color = "black") +
  geom_vline(aes(xintercept = mean(xeFG)), color = "black") +
  # geom_abline(intercept = 222, slope = -1, color = "black") +
  # geom_abline(slope = -1,color="black")+
  geom_image(
    aes(
      x = xeFG, y = eFG,
      image = urlThumbnailTeam
    ),
    size = 0.1
  ) +
  # geom_text(nudge_x = 1.3, nudge_y = 0, size = 6,check_overlap = TRUE) +
  # geom_text_repel(nudge_x = 1, nudge_y = 0.5,size=6,min.segment.length=10) +
  # geom_label(nudge_x = 1.3, nudge_y = 0, size = 6) +
  theme_owen() +
  theme(
    plot.title.position = "plot",
    plot.title = element_text(face = "bold", size = 24, hjust = 0.5),
    plot.margin = margin(10, 10, 15, 10),
    plot.subtitle = element_text(size = 18),
    plot.caption = element_text(size = 14)
  ) +
  theme(
    axis.text.x = element_text(size = 14, face = "bold", color = "black"),
    axis.text.y = element_text(size = 14, face = "bold", color = "black"),
    axis.title.x = element_text(size = 18, face = "bold", colour = "black"),
    axis.title.y = element_text(size = 18, face = "bold", colour = "black")
  ) +
  scale_y_continuous(labels=scales::percent) +
  scale_x_continuous(labels=scales::percent) +
  labs(
    title = paste0("eFG% Actual vs Expected"),
    x = "Expected eFG%", y = "eFG%",
    # subtitle = "Net Ratings here are adjusted for Strength of Schedule | Last 10 Games Only",
    subtitle = "According to SSQM v2.0, Shot Making: PPS above league avg",
    caption = "@sradjoker | source: nba.com/stats"
  )
ggsave("../figs/team_leaders/team_eFG_xeFG.png", p, w = 10 * 1.5, h = 8 * 1.5, dpi = 300)